In [ ]:
# Load relevant packages

import pandas as pd
import numpy as np
import re
import csv
import operator
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from matplotlib import pyplot as plt
from collections import defaultdict, Counter

In [ ]:
df = pd.read_csv("founta2018_formative2.csv")

In [ ]:
df.sample(5, random_state = 123)

,tweet,label
42083,MK Dons Boss Robbie Neilson Hails Leicester Yo...,normal
72731,EXO's Sehun shares future plans for upcoming s...,normal
71150,What if I've recharged my number with 303 on 2...,normal
50169,Nice Move-in ready House with great views in w...,spam
36734,I wish I was an ear. They literally need to ea...,normal


In [ ]:
def clean_text(text):

    #replace mentions and URLs with special token
       text = re.sub(r"@[A-Za-z0-9_-]+",'USR',text)
       text = re.sub(r"http\S+",'URL',text)
    
    # remove newline and tab characters
       text = text.replace('\n',' ')
       text = text.replace('\t',' ')
       text = text.replace('rt', ' ')

       text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    
    # strip whitespace
       text = text.strip()
    
    # lowercase
       text = text.lower()
    
       return text

In [ ]:
# Clean tweets
df["tweet"] = df.tweet.apply(lambda x: clean_text(x))

In [ ]:
def drop_dupl(df):

    # save number of documents before dropping duplicates
    n_docs = df.shape[0]

    # drop duplicates
    df.drop_duplicates(subset = ['tweet'], inplace=True)

    print(f'{n_docs} posts, of which {n_docs - df.shape[0]} were dropped for being duplicates.')
    print(f'{df.shape[0]} posts remain. \n')
    
    return df

df = drop_dupl(df)

99996 posts, of which 12710 were dropped for being duplicates.
87286 posts remain. 



In [ ]:
with open('FastTextDictionary.pickle', 'rb') as handle:
    FastText = pickle.load(handle)

In [ ]:
def lis(text):
  return text.split(' ')

In [ ]:
df['cleaned'] = df.tweet.apply(lis)

In [ ]:
df.head(5)

,tweet,label,cleaned
1,rt usr: man it would fucking rule if we had a ...,non-hateful,"[rt, usr:, man, it, would, fucking, rule, if, ..."
2,"it is time to draw close to him father, i dr...",non-hateful,"[it, is, time, to, draw, close, to, him, , , f..."
3,if you notice me sta to act different or dist...,non-hateful,"[if, you, notice, me, sta, , to, act, differen..."
4,"forget unfollowers, i believe in growing. 7 ne...",non-hateful,"[forget, unfollowers,, i, believe, in, growing..."
5,rt usr: hate being sexually frustrated like i ...,non-hateful,"[rt, usr:, hate, being, sexually, frustrated, ..."


In [ ]:
# Split data into training, development, and test sets
train, dev_test = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=0)
dev, test = train_test_split(dev_test, test_size=0.5, stratify=dev_test['label'], random_state=0)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pickle
import numpy as np
import pandas as pd
import torch
import warnings

from collections import defaultdict, Counter
from string import punctuation
from matplotlib import pyplot as plt
from nltk.util import bigrams
from tqdm import tqdm

from sklearn.feature_extraction import _stop_words
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertModel, BertTokenizer

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Define dataset class
class BERTDataset(Dataset):

    def __init__(self, data):
        
        # Initialize tokenizer
        self.tok = BertTokenizer.from_pretrained('bert-base-uncased')
        
        # Truncate and encode abstracts
        self.tweets = (data.cleaned.apply(self.tok.encode, max_length=50, truncation=True))
        
        # Store labels
        self.labels = list(data.label)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        tweet = self.tweets[idx]
        label = self.labels[idx]
        return tweet, label

In [ ]:
def bert_collate(batch):
    # Store batch size
    batch_size = len(batch)
    
    # Separate tweets and labels
    tweets = [t for t, l in batch]
    labels = [l for t, l in batch]
    
    # Check that all labels are valid
    valid_labels = {"hateful", "normal", "abusive", "spam"}
    if not set(labels).issubset(valid_labels):
        raise ValueError("Invalid label found in batch: {}".format(labels))
    
    # Convert labels to integers
    label2id = {"hateful": 0, "normal": 1, "abusive" : 2, "spam": 3}
    label_ids = torch.tensor([label2id[l] for l in labels]).long()
    
    # Store length of longest tweet in batch
    max_len = max(len(t) for t in tweets)
    
    # Create padded tweet and attention mask tensors
    tweets_pad = torch.zeros((batch_size, max_len)).long()
    masks_pad = torch.zeros((batch_size, max_len)).long()
    for i, t in enumerate(tweets):
        tweets_pad[i, :len(t)] = torch.tensor(t)
        masks_pad[i, :len(t)] = 1
    
    return tweets_pad, masks_pad, label_ids

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassifier class

class BertClassifier_RELU(nn.Module):
    
    def __init__(self, freeze_bert=False):
        super(BertClassifier,self).__init__()
        
        # Specify hidden size of Bert, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50 , 4
        
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        
        self.classifier = nn.Sequential(
                            nn.Linear(D_in, H),
                            nn.ReLU(),
                            nn.Linear(H, D_out))
        
        # Freeze the Bert Model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
    
    def forward(self,input_ids,attention_mask, output_attentions=True):

        outputs = self.bert(input_ids=input_ids,
                           attention_mask = attention_mask, output_attentions=True)
        
        overall = outputs

        attentions = outputs.attentions
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:,0,:]
        
        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)
        
        return logits, overall, attentions

In [ ]:
# Define BERT classifier
class BERTClassifier(nn.Module):

    def __init__(self):
        
        # Define network layers
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.linear = nn.Linear(768, 4)
        
        # Define dropout
        self.dropout = nn.Dropout(0.2)
        
        # Freeze BERT layers
        for n, p in self.bert.named_parameters():
            p.requires_grad = False

    def forward(self, tweets, masks, output_attentions=True):
        
        # Define flow of tensors through network
        outputs = self.bert(tweets, attention_mask=masks, output_attentions = True)
        attentions = outputs.attentions
        output_bert = self.bert(tweets, attention_mask=masks)[0].mean(axis=1)
        return self.linear(self.dropout(output_bert)), outputs, attentions

In [ ]:
train = train.reset_index()
dev = dev.reset_index()
test = test.reset_index()

In [ ]:
# Create datasets
train_dataset = BERTDataset(train)
dev_dataset = BERTDataset(dev)
test_dataset = BERTDataset(test)

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=100, collate_fn=bert_collate, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=100, collate_fn=bert_collate)
test_loader = DataLoader(test_dataset, batch_size=100, collate_fn=bert_collate)

In [ ]:
# Initialize model
model = BERTClassifier()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!pip install sadice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sadice import SelfAdjDiceLoss

criterion = SelfAdjDiceLoss()

In [ ]:
import torch.nn as nn

class F1Loss(nn.Module):
    def __init__(self, eps=1e-7):
        super().__init__()
        self.eps = eps

    def forward(self, y_pred, y_true):
        tp = (y_pred * y_true).sum(dim=0)
        fp = (y_pred * (1 - y_true)).sum(dim=0)
        fn = ((1 - y_pred) * y_true).sum(dim=0)

        precision = tp / (tp + fp + self.eps)
        recall = tp / (tp + fn + self.eps)
        f1 = 2 * (precision * recall) / (precision + recall + self.eps)

        return 1 - f1.mean()

In [ ]:
import torch.nn as nn

class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, weights=None):
        super().__init__()
        self.weights = weights

    def forward(self, input, target):
        log_probs = nn.functional.log_softmax(input, dim=1)
        loss = nn.functional.nll_loss(log_probs, target, weight=self.weights)
        return loss

In [ ]:
# Define optimizer and training objective
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [ ]:
print(len(dev_loader))

88


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Train model
for e in range(4):

    model.train()

    for i, b in enumerate(tqdm(train_loader)):

        # Perform forward pass
        optimizer.zero_grad()
        tweets, masks, lbls = [t for t in b]
        output,overall,attn = model(tweets, masks, output_attentions= True)
        loss = criterion(output, lbls)
        
        # Perform backpropagation and update weights
        loss.backward()
        optimizer.step()
  
    # Evaluate model on development data
    model.eval()

    y_true = list()
    y_pred = list()

    with torch.no_grad():
        for b in dev_loader:
            tweets, masks, lbls = [t for t in b]
            output,overall,attn = model(tweets, masks)
            max_output = output.argmax(dim=1)
            y_true.extend(lbls.tolist())
            y_pred.extend(max_output.tolist())

    print("Finished epoch", e+1)

    print('Accuracy after {} epoch(s): {:.2f}'.format(e+1, accuracy_score(y_true, y_pred)))
    print(confusion_matrix(y_true,y_pred))
    print(classification_report(y_true,y_pred))

100%|██████████| 699/699 [24:01<00:00,  2.06s/it]


Finished epoch 1
Accuracy after 1 epoch(s): 0.74
[[  58  263   79    7]
 [  20 4568   60  410]
 [ 108  530 1322   53]
 [   0  656   50  545]]
              precision    recall  f1-score   support

           0       0.31      0.14      0.20       407
           1       0.76      0.90      0.82      5058
           2       0.87      0.66      0.75      2013
           3       0.54      0.44      0.48      1251

    accuracy                           0.74      8729
   macro avg       0.62      0.53      0.56      8729
weighted avg       0.73      0.74      0.73      8729



100%|██████████| 699/699 [23:55<00:00,  2.05s/it]


Finished epoch 2
Accuracy after 2 epoch(s): 0.76
[[  24  155  222    6]
 [  19 4251  302  486]
 [  10  187 1790   26]
 [   0  601   80  570]]
              precision    recall  f1-score   support

           0       0.45      0.06      0.10       407
           1       0.82      0.84      0.83      5058
           2       0.75      0.89      0.81      2013
           3       0.52      0.46      0.49      1251

    accuracy                           0.76      8729
   macro avg       0.64      0.56      0.56      8729
weighted avg       0.74      0.76      0.74      8729



100%|██████████| 699/699 [24:00<00:00,  2.06s/it]


Finished epoch 3
Accuracy after 3 epoch(s): 0.77
[[   7  224  174    2]
 [   5 4797  161   95]
 [   5  330 1672    6]
 [   0  982   62  207]]
              precision    recall  f1-score   support

           0       0.41      0.02      0.03       407
           1       0.76      0.95      0.84      5058
           2       0.81      0.83      0.82      2013
           3       0.67      0.17      0.27      1251

    accuracy                           0.77      8729
   macro avg       0.66      0.49      0.49      8729
weighted avg       0.74      0.77      0.72      8729



100%|██████████| 699/699 [23:52<00:00,  2.05s/it]


Finished epoch 4
Accuracy after 4 epoch(s): 0.76
[[  21  246  138    2]
 [   8 4824   99  127]
 [  23  422 1556   12]
 [   0  941   56  254]]
              precision    recall  f1-score   support

           0       0.40      0.05      0.09       407
           1       0.75      0.95      0.84      5058
           2       0.84      0.77      0.81      2013
           3       0.64      0.20      0.31      1251

    accuracy                           0.76      8729
   macro avg       0.66      0.50      0.51      8729
weighted avg       0.74      0.76      0.72      8729



In [ ]:
torch.save(model.state_dict(), 'bert_finding_attention_50len.pth')

In [ ]:
from transformers import BertTokenizer

# Instantiate tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def display_sentence_attention(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
    tensor_tokens = torch.tensor([indexed_tokens])

    # Create attention masks
    attention_mask = [1] * len(tokens)

    # Convert inputs to PyTorch tensors
    tensor_mask = torch.tensor([attention_mask])

    # Predict label and get attention scores
    with torch.no_grad():
        #outputs, attn_scores = model(tensor_tokens, tensor_mask, return_dict=False, output_attentions=True)
        outputs, all, attn_scores = model(tensor_tokens, tensor_mask, output_attentions=True)
        pred_label = torch.argmax(outputs).item()

    # Decode attention scores and display highlighted sentence
    attn_scores = _decode_output({'attentions': attn_scores})
    display(HTML(f'<strong>Predicted Label:</strong> {pred_label}'))
    for layer in range(len(attn_scores['aa']['attn'])):
        for head in range(len(attn_scores['aa']['attn'][layer])):
            display(HTML(f'<strong>Layer {layer+1} Head {head+1}:</strong>'))
            disp_attn_tokens(tokens, attn_scores['aa']['attn'], layer, head)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
import html

In [ ]:
import html

def display_attention(tokens, attn_matrix, layer=-1, head=0, direction='ba'):
    token_weights = attn_matrix[layer][head].detach().cpu().numpy()
    tokens = [t for t in tokens if t != tokenizer.pad_token]
    attention_weights = token_weights[direction == 'ba'][0]
    highlighted_text = []
    for i, (token, weight) in enumerate(zip(tokens, attention_weights)):
        if weight != 0:
            # Calculate the background color for the current token based on its attention weight
            color = f'rgba(255,0,0,{weight:.8f})'
            # If this isn't the first token in the sentence, calculate the color for the previous token as well
            if i > 0:
                prev_weight = attention_weights[i-1]
                if prev_weight != 0:
                    prev_color = f'rgba(255,0,0,{prev_weight:.8f})'
                else:
                    prev_color = 'white'
            else:
                prev_color = 'white'
            # Add the current token to the highlighted text string with a gradient background color
            highlighted_text.append(f'<span style="background: linear-gradient(to right, {prev_color}, {color})">{html.escape(str(token))}</span>')
        else:
            highlighted_text.append(str(token))
    highlighted_text = ' '.join(highlighted_text)
    display(HTML(highlighted_text))

In [ ]:
def display_attention_2(tokens, attn_matrix, layer=-1, head=0, direction='ba'):
    token_weights = attn_matrix[layer][head].detach().cpu().numpy()
    attention_weights = token_weights[direction == 'ba'][0][:len(tokens)]
    tokens = [t for t in tokens if t != tokenizer.pad_token]
    highlighted_text = []
    for i, (token, weight) in enumerate(zip(tokens, attention_weights)):
        color = f'rgba(255,0,0,{weight:.2f})'
        highlighted_text.append(f'<span style="background-color: {color}">{html.escape(token)}</span>')
    highlighted_text = ' '.join(highlighted_text)
    display(HTML(highlighted_text))


In [ ]:

input_text = "If you're a black person, kill yourself!!!! url"
tokens = tokenizer.encode_plus(input_text, max_length=128, truncation=True, padding='max_length', return_tensors='pt')

# Get BERT model predictions and attentions
predictions, outputs, attentions = model(tokens['input_ids'], tokens['attention_mask'])


# Reshape attention tensor
n_heads = attentions[11].shape[1]
attn_matrix = attentions[-1].reshape(n_heads, tokens['input_ids'].shape[1], tokens['input_ids'].shape[1])

decoded_tokens = tokenizer.decode(tokens['input_ids'][0])
decoded_tokens = decoded_tokens.split(' ')

display_attention(decoded_tokens, attn_matrix, layer= 1, head=11)


In [ ]:
df['tweet'][df['label']=="hateful"].sample(1)

64557    the improve troupe changed the names of all th...
Name: tweet, dtype: object

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()

y_true = list()
y_pred = list()

with torch.no_grad():
    for b in test_loader:
        tweets, masks, lbls = [t for t in b]
        output, overall, attn = model(tweets, masks)
        max_output = output.argmax(dim=1)
        y_true.extend(lbls.tolist())
        y_pred.extend(max_output.tolist())

print('Test accuracy: {:.2f}'.format(accuracy_score(y_true, y_pred)))
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true, y_pred))

Test accuracy: 0.77
[[  18  243  144    2]
 [   9 4815  108  126]
 [  20  407 1577   10]
 [   0  922   49  279]]
              precision    recall  f1-score   support

           0       0.38      0.04      0.08       407
           1       0.75      0.95      0.84      5058
           2       0.84      0.78      0.81      2014
           3       0.67      0.22      0.33      1250

    accuracy                           0.77      8729
   macro avg       0.66      0.50      0.52      8729
weighted avg       0.74      0.77      0.73      8729



In [ ]:
schema = {0:"hateful",1:"normal", 2:"abusive", 3:"spam"}
test['pred'] = [schema[i] for i in y_pred]

In [ ]:
check = test[test['label']=="hateful"]
check = check.reset_index()
check[check["pred"]=="hateful"].sample(5)

,level_0,index,tweet,label,cleaned,pred
184,4209,22183,usr usr too bad he's a xenophobic idiot with t...,hateful,"[usr, usr, too, bad, he's, a, xenophobic, idio...",hateful
249,5258,73986,rt usr: i'm embarrassed for those that voted f...,hateful,"[rt, usr:, i'm, embarrassed, for, those, that,...",hateful
312,6531,27101,professor: republicans criticize susan rice be...,hateful,"[professor:, republicans, criticize, susan, ri...",hateful
218,4769,78749,rt usr: you know niggas hate water smh url,hateful,"[rt, usr:, you, know, niggas, hate, water, smh...",hateful
93,2114,83496,"usr usr he is rubbish, a subhuman idiot ! him ...",hateful,"[usr, usr, he, is, rubbish,, a, subhuman, idio...",hateful


In [ ]:
check['tweet'].iloc[184]

"usr usr too bad he's a xenophobic idiot with the reading level of a 3rd grader with racist parents"

In [ ]:
train[train['label']=="hateful"].sample(5)

,index,tweet,label,cleaned
64845,43829,don't hate you because niggas can't appreciate...,hateful,"[don't, hate, you, because, niggas, can't, app..."
10558,72500,usr usr usr plenty of rats and rabbits are kil...,hateful,"[usr, usr, usr, plenty, of, rats, and, rabbits..."
8387,27095,rt usr: nigga u crazy url,hateful,"[rt, usr:, nigga, u, crazy, url]"
18479,79835,i question whether or not the filthy casuals w...,hateful,"[i, question, whether, or, not, the, filthy, c..."
36941,96195,anybody that does not obey the teachings of th...,hateful,"[anybody, that, does, not, obey, the, teaching..."


In [ ]:
train['tweet'].iloc[36941]

'anybody that does not obey the teachings of the bible and jesus christ is a child of the devil a worker of iniquity url'